In [ ]:
import numpy as np
import pyvista as pv
import pickle
import matplotlib.pyplot as plt
import colorcet as cc
import xarray as xr

from kale.algorithms import contour_banded
from kale import theme

%config InlineBackend.figure_format = "retina"

# Load Data

In [ ]:
MESH_GEOMETRY_FILE_NAME = "nankai_121728_clean.vtk"
MESH_VALUES_FILE_NAME = "2023_03_04_00_02_03.hdf"
MESH_Z_SCALE = 0.025
TIME_STEP_INTERESTING = 1000

In [ ]:
ds = xr.open_dataset(MESH_VALUES_FILE_NAME, engine="netcdf4")
data = ds["loading_rate"][0, :]
mesh = pv.read(MESH_GEOMETRY_FILE_NAME)
mesh.cell_data["data"] = data
mesh.points[:, -1] *= MESH_Z_SCALE

# Extract boundary of mesh
boundary = mesh.extract_feature_edges(
    boundary_edges=True,
    non_manifold_edges=False,
    feature_edges=False,
    manifold_edges=False,
)
boundary.clear_data()

# Load fields from .npz file
npz_data = np.load("skies_single_step_erosion_figures_nankai.npz")
mesh.cell_data["pre_event_slip_deficit"] = npz_data["name1"]
mesh.cell_data["event_slip"] = npz_data["name2"]
mesh.cell_data["post_event_slip_deficit"] = npz_data["name3"]
mesh.cell_data["meshes_areas"] = npz_data["name4"]

# Read pickle file with interesting times
time_series = pickle.load(open("time_series.pickle", "rb"))

In [ ]:
start_idx = 0
end_idx = len(time_series.time)
plt.fill_between(
    time_series.time[start_idx:end_idx],
    # real_time[start_idx:end_idx],
    np.zeros_like(time_series.time[start_idx:end_idx]),
    # np.zeros_like(real_time[start_idx:end_idx]),
    time_series.probability_weight[start_idx:end_idx],
    color="gold",
    alpha=1.0,
    edgecolor=None,
)
plt.show()

# This is the Matplotlib plot to combine with the banded contour plot below into a single figure

In [ ]:
time_series
start_idx = 0
end_idx = len(time_series.time)
minimum_event_moment_magnitude = 5.0

idxs = np.arange(0, len(time_series.time))
real_time = idxs * 50 * 5e-7 * 1000

event_idx = np.where(time_series.event_trigger_flag == 1)[0]
# figsize = (10, 3)
# plt.figure(figsize=figsize)

figsize = (9, 5)
plt.figure(figsize=figsize)

# Plot probability time series
plt.subplot(2, 1, 1)
plt.plot(
    real_time[start_idx:end_idx],
    time_series.probability_weight[start_idx:end_idx],
    "-k",
    linewidth=0.5,
)
plt.plot(
    # [time_series.time[start_idx], time_series.time[end_idx - 1]],
    [real_time[start_idx], real_time[end_idx - 1]],
    [0, 0],
    "-k",
    linewidth=0.5,
)
plt.fill_between(
    # time_series.time[start_idx:end_idx],
    real_time[start_idx:end_idx],
    # np.zeros_like(time_series.time[start_idx:end_idx]),
    np.zeros_like(real_time[start_idx:end_idx]),
    time_series.probability_weight[start_idx:end_idx],
    color="gold",
    alpha=1.0,
    edgecolor=None,
)

plt.xlim([real_time[start_idx], real_time[end_idx - 1]])
plt.gca().set_ylim(bottom=0.0)
plt.ylabel("$p^t$")


# Plot earthquake magnitude stem plot
plt.subplot(2, 1, 2)

for i in range(event_idx.size):
    plt.plot(
        [
            real_time[event_idx[i]],  # time_series.time[event_idx[i]],
            real_time[event_idx[i]],  # time_series.time[event_idx[i]],
        ],
        [
            minimum_event_moment_magnitude,
            time_series.event_magnitude[event_idx[i]],
        ],
        "-",
        linewidth=0.1,
        zorder=10,
        color="k",
    )

cmap = cc.cm.CET_L17
magnitude_plot_size = 1e-5 * 9 ** time_series.event_magnitude[event_idx]
plt.scatter(
    # time_series.time[event_idx],
    real_time[event_idx],
    time_series.event_magnitude[event_idx],
    s=magnitude_plot_size,
    c=time_series.event_magnitude[event_idx],
    zorder=20,
    alpha=1.0,
    edgecolors="k",
    linewidths=0.5,
    cmap=cmap,
    vmin=6.0,
    vmax=9.0,
)

plt.xlabel("time (years)")
plt.ylabel("$\mathrm{M}_\mathrm{W}$")
# plt.xlim([start_idx, end_idx])
plt.xlim([real_time[start_idx], real_time[end_idx - 1]])
plt.gca().set_ylim(bottom=5.5)
plt.gca().set_ylim([5.5, 9.0])
plt.tight_layout()
plt.savefig("time_series_real_time.pdf", dpi=500)

In [ ]:
print(time_series.event_trigger_flag)

time_series.event_magnitude[event_idx].min()
time_series.event_magnitude[event_idx].min()
mesh

# Banded contour for a single time

In [ ]:
pl = pv.Plotter()
pl.add_mesh(boundary)
pl.add_floor("-z", show_edges=True, edge_color="white", color="lightgray")
pl.image_scale = 2
CONTOUR_LEVELS = np.linspace(-60, 60, 21)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]
contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="data",
)
pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars="data",
    n_colors=N_COLORS,
    show_scalar_bar=True,
    scalar_bar_args=dict(title=f"", **theme.SCALAR_BAR_OPTS),
)
pl.add_mesh(edges)
pl.enable_ssao(radius=2, bias=0.5)
pl.enable_anti_aliasing("ssaa")
# pl.camera.view_angle = 150
pl.show()

In [ ]:
pl.parallel_projection

In [ ]:
CONTOUR_LEVELS = np.linspace(-10, 10, 21)
N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]
CAMERA_ZOOM = 1.0
IMAGE_SCALE = 4
TITLE_POSITION = [0.1, 0.62]
CAMERA_POSITION = [
    (132.7456379797715, 53.03872370660612, 35.03423385946435),
    (134.73400115966797, 33.14949893951416, -0.625),
    (-0.2950474880132869, -0.8413651518097409, 0.45282630349530384),
]

pl = pv.Plotter(shape=(1, 3), border=False, multi_samples=8, line_smoothing=True)

labels = ["pre-event", "event", "post-event"]
for i in range(3):
    pl.subplot(0, i)
    pl.add_mesh(boundary)
    # box = pv.Box(mesh.bounds)
    # pl.add_mesh(box, color="lightgrey", culling="front")
    pl.add_floor("-z", show_edges=True, edge_color="white", color="lightgray")
    pl.add_text(
        labels[i],
        position=TITLE_POSITION,
        color="k",
        shadow=False,
        font_size=12,
        viewport=True,
    )

annotations = {
    0.0: "geometric moment / area (m)",
}

# Pre-event geometric moment
pl.subplot(0, 0)
contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="pre_event_slip_deficit",
)
pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars="pre_event_slip_deficit",
    n_colors=N_COLORS,
)
pl.add_mesh(edges)

# Event - Coseismic geometric moment
pl.subplot(0, 1)
contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="event_slip",
)
pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars="event_slip",
    n_colors=N_COLORS,
    show_scalar_bar=True,
    scalar_bar_args=dict(title=f" ", **theme.SCALAR_BAR_H),
    annotations=annotations,
)
pl.add_mesh(edges)

# Post-event slip
pl.subplot(0, 2)
contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="post_event_slip_deficit",
)
pl.add_mesh(
    contour,
    cmap="RdYlBu_r",
    clim=CLIM,
    scalars="post_event_slip_deficit",
    n_colors=N_COLORS,
)
pl.add_mesh(edges)


# pl.enable_ssao(radius=15, bias=0.5)
# pl.enable_anti_aliasing('ssaa')
# pl.camera.zoom(1.7)
# pl.enable_shadows()

pl.link_views()
pl.camera_position = CAMERA_POSITION
pl.camera.zoom(CAMERA_ZOOM)

pl.image_scale = IMAGE_SCALE

pl.show()
print(pl.camera_position)
pl.screenshot("kale_single_step_erosion_figures_nankai.png", return_img=False)

# Plot a single time step from the experiment
1. Loading rate
2. Moment
3. Cumulative slip
# ['cumulative_slip', 'geometric_moment', 'last_event_slip', 'loading_rate']

In [ ]:
# idx = 45000
# mesh.cell_data["geometric_moment"] = ds["geometric_moment"][idx, :] # / mesh.cell_data["meshes_areas"]
# mesh.cell_data["last_event_slip"] = ds["last_event_slip"][idx, :]
# mesh.cell_data["cumulative_slip"] = ds["cumulative_slip"][idx, :]

CONTOUR_LEVELS = np.linspace(0, 15, 11)
# CONTOUR_LEVELS = np.logspace(-1, 1, 11, endpoint=True)
# CONTOUR_LEVELS = np.array([0.1, 0.5, 1.0, 3.0, 5.0, 10.0, 15.0])

CONTOUR_LINE_WIDTH = 5

N_COLORS = len(CONTOUR_LEVELS) - 1
CLIM = [np.min(CONTOUR_LEVELS), np.max(CONTOUR_LEVELS)]
CAMERA_ZOOM = 1.0
IMAGE_SCALE = 14  # See https://github.com/pyvista/pyvista/issues/4296
TITLE_POSITION = [0.1, 0.62]
CAMERA_POSITION = [
    (132.7456379797715, 53.03872370660612, 35.03423385946435),
    (134.73400115966797, 33.14949893951416, -0.625),
    (-0.2950474880132869, -0.8413651518097409, 0.45282630349530384),
]

pl = pv.Plotter(shape=(1, 3), border=False, multi_samples=8, line_smoothing=True)

for i in range(3):
    pl.subplot(0, i)
    pl.add_mesh(boundary)
    # box = pv.Box(mesh.bounds)
    # pl.add_mesh(box, color="lightgrey", culling="front")
    pl.add_floor("-z", show_edges=True, edge_color="white", color="lightgray")

annotations = {
    7.5: "cumulative slip (m)",
}

pl.subplot(0, 0)
# Time step 1: Cumulative slip
idx = 10000
mesh.cell_data["cumulative_slip_1"] = ds["cumulative_slip"][idx, :]

contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="cumulative_slip_1",
)
pl.add_mesh(
    contour,
    cmap="CET_L17",
    clim=CLIM,
    scalars="cumulative_slip_1",
    n_colors=N_COLORS,
)
pl.add_mesh(edges)
pl.add_text(
    # f"i={str(idx)}",
    f"{real_time[idx]:0.2f} (years)",
    position=TITLE_POSITION,
    color="k",
    shadow=False,
    font_size=12,
    viewport=True,
)

pl.subplot(0, 1)
# Time step 2: Cumulative slip
# Plotting this first because it seems to enable
# colorbar in the center
idx = 17000
mesh.cell_data["cumulative_slip_2"] = ds["cumulative_slip"][idx, :]

contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="cumulative_slip_2",
)
pl.add_mesh(
    contour,
    cmap="CET_L17",
    clim=CLIM,
    scalars="cumulative_slip_2",
    n_colors=N_COLORS,
    show_scalar_bar=True,
    scalar_bar_args=dict(title=f" ", **theme.SCALAR_BAR_H),
    annotations=annotations,
)
pl.add_mesh(edges)
pl.add_text(
    # f"i={str(idx)}",
    f"{real_time[idx]:0.2f} (years)",
    position=TITLE_POSITION,
    color="k",
    shadow=False,
    font_size=12,
    viewport=True,
)

pl.subplot(0, 2)
# Time step 3: Cumulative slip
idx = 24000
mesh.cell_data["cumulative_slip_3"] = ds["cumulative_slip"][idx, :]

contour, edges = contour_banded(
    mesh,
    CONTOUR_LEVELS,
    rng=CLIM,
    scalars="cumulative_slip_3",
)
pl.add_mesh(
    contour,
    cmap="CET_L17",
    clim=CLIM,
    scalars="cumulative_slip_3",
    n_colors=N_COLORS,
)
pl.add_mesh(edges)
pl.add_text(
    # f"i={str(idx)}",
    f"{real_time[idx]:0.2f} (years)",
    position=TITLE_POSITION,
    color="k",
    shadow=False,
    font_size=12,
    viewport=True,
)

# pl.enable_ssao(radius=15, bias=0.5)
# pl.enable_anti_aliasing('ssaa')
# pl.camera.zoom(1.7)

pl.link_views()
pl.camera_position = CAMERA_POSITION
pl.camera.zoom(CAMERA_ZOOM)

pl.image_scale = IMAGE_SCALE

pl.show()
pl.screenshot(
    "kale_single_time_step_cumulative_slip_nankai_years_large.png",
    return_img=False,
)